Data needed in input:
- Chords extracted from segmentation masks of epifluorescence timelapses (.npy archives, here called `cld_bulk_thermal_ramps.npy`) in `thermal_ramps`. To obtain these: 
    - First get segmentation masks from epifluorescence timelapses by using the FIJI macro "Fiji utils/RNA_epifluorescence_timelapse_segmentation.ijm"). 
    - Then, perform CLD extraction using the notebook at "CLD/CLD_from_Binary_Masks.ipynb

In [ ]:
# Import dependencies
import matplotlib.patheffects as pe
import matplotlib.pyplot as plt
import numpy as np
import os
import porespy as ps
import seaborn as sns

from scipy.optimize import curve_fit
from skimage.measure import regionprops
from tqdm.notebook import tqdm

In [ ]:
# Replace with absolute path to folder containing the .npy file of chord lengths
# extracted from thermal ramps CLD extraction (using `CLD/CLD_from_Binary_Masks.ipynb`).
# In this script, the resulting file is named `cld_bulk_thermal_ramps.npy'.
thermal_ramps = "/ABSOLUTE/PATH/TO/CHORDS/NPY/FILE/"

In [ ]:
def cld_to_chords_unj(cld_dict, binning = 1): 
    # cld_dict = input dictionary containing chords_x and chords_y per sample repeat vs time
    # binning = if images (and masks) have been binned, change binning factor -- default is 1 
    # (image analysed in original format and resolution, 2044px x 2048px)
    # Initialise dictionary
    chords_vs_time_unj = {}
    # Chord Lengths are in px -> need to convert to um to extract physical size information
    px_um_conv = 3.0852 # px/um for 20x lens used on Nikon Ti2
    # If we have binning, we need to adjust this px_um_conv factor
    pxum_conv_bin = px_um_conv/binning 
    # Looping through samples in experiment
    for ind, sample in enumerate(cld_dict.keys()): 
        print(sample)
        # Initialising inner sample list
        chords_vs_time_unj[sample] = {}
        # Looping through repeats - keeping repeats separate
        if len(list(cld_dict[sample].keys())) == 3: 
            for repeat in cld_dict[sample].keys(): 
                chords_vs_time_unj[sample][repeat] = []
                # Looping through timepoints
                for timepoint in tqdm(range(len(cld_dict[sample][1]['count_x']))):
                    chords_xy = (1/pxum_conv_bin)*np.concatenate(
                        (cld_dict[sample][repeat]['count_x'][timepoint],
                         cld_dict[sample][repeat]['count_y'][timepoint])
                    )
                    chords_vs_time_unj[sample][repeat].append(list(chords_xy))
        # ...unless they refer to the same FOV, but different channels -- i.e. RNA nanostar C
        elif len(list(cld_dict[sample].keys())) == 6: 
            print('More than repeats - will merge 1-4, 2-5, 3-6')
            for repeat in list(cld_dict[sample].keys())[:3]: 
                chords_vs_time_unj[sample][repeat] = []
                # Looping through timepoints
                for timepoint in tqdm(range(len(cld_dict[sample][1]['count_x']))):
                    chords_xy1 = (1/pxum_conv_bin)*np.concatenate(
                        (cld_dict[sample][repeat]['count_x'][timepoint], 
                         cld_dict[sample][repeat]['count_y'][timepoint])
                    )
                    chords_xy2 = (1/pxum_conv_bin)*np.concatenate(
                        (cld_dict[sample][repeat+3]['count_x'][timepoint], 
                         cld_dict[sample][repeat+3]['count_y'][timepoint])
                    )
                    chords_vs_time_unj[sample][repeat].append(list(chords_xy1)+list(chords_xy2))
    return chords_vs_time_unj

In [ ]:
# Auxiliary Functions
def sigmoidify(arr): 
    arr2 = arr.copy()
    midpoint = (np.max(arr) + np.min(arr))/2
    for i in range(len(arr2)): 
        if arr2[i] >= midpoint: 
            arr2[i] = 1
        else:
            arr2[i] = 0
    return arr[0]*arr2

def get_step(step_func): 
    for i in range(len(step_func)): 
        if step_func[i] > 0 and step_func[i+1] == 0: 
            return i

In [ ]:
# Move to correct directory and load the serialised chord lengths
os.chdir(thermal_ramps)
cld_thermal_ramps = np.load('cld_bulk_thermal_ramps.npy', allow_pickle = True).item()

In [ ]:
# Preprocess to obtain joined X and Y chord lengths for all repeats individually
# In the case of sample C, for which the analysis is carried out in both MG and DFHBI
# channels, the chords from matching channels of the same FOV are concatenated together
chords_melt_bulk_unj = cld_to_chords_unj(cld_thermal_ramps)

In [ ]:
# Extract mean and standard deviation vs time for all sample repeats
mean_melt_bulk_unj, std_melt_bulk_unj = {}, {}
# Loop through samples in dictionary keys
for sample in chords_melt_bulk_unj.keys(): 
    # Initialise blank timepoint-spanning lists within the output dictionaries
    mean_melt_bulk_unj[sample], std_melt_bulk_unj[sample] = {}, {}
    # Loop through repeats
    for repeat in tqdm(range(1, 1+len(chords_melt_bulk_unj[sample].keys()))):
        #print(repeat)
        mean_melt_bulk_unj[sample][repeat], std_melt_bulk_unj[sample][repeat] = [], []
        # Loop through timepoints - one CLD per sample per timepoint
        for timepoint in range(len(chords_melt_bulk_unj[sample][repeat])):
            mean_melt_bulk_unj[sample][repeat].append(np.mean(chords_melt_bulk_unj[sample][repeat][timepoint]))
            std_melt_bulk_unj[sample][repeat].append(np.std(chords_melt_bulk_unj[sample][repeat][timepoint], ddof = 1))

In [ ]:
# Combine information from all repeats to yield a single mean and standard error arrays per sample
mean_melt_bulk_comb, std_melt_bulk_comb = {}, {}
# Loop through samples in dictionary keys
for sample in mean_melt_bulk_unj.keys(): 
    # Initialise blank timepoint-spanning lists within the output dictionaries
    means_list = [
        np.array(mean_melt_bulk_unj[sample][repeat]) 
        for repeat in list(mean_melt_bulk_unj[sample].keys())
    ]
    mean_melt_bulk_comb[sample] = np.mean(means_list, axis = 0)
    std_melt_bulk_comb[sample] = np.std(means_list, axis = 0, ddof = 1)

In [ ]:
# Figure S8b -- melting profiles and T_M calculation
temp = np.array(list(range(25, 76)))
colours = {'NS_A' : 'orangered', 'NS_B' : 'cyan', 'NS_C' : 'gray'}
labels = {'NS_A' : 'A', 'NS_B' : 'B', 'NS_C' : 'C'}

plt.subplots(3, 1, figsize = (3, 9), sharey = True)
plt.subplots_adjust(hspace= .1)
for sample, ind in zip(mean_melt_bulk_comb.keys(), range(len(mean_melt_bulk_comb.keys()))): 
    plt.subplot(3, 1, ind+1)
    plt.plot(temp, 
             np.nan_to_num(mean_melt_bulk_comb[sample]), 
             lw = 2.0, label = labels[sample], color = colours[sample])
    plt.fill_between(temp, 
                     np.nan_to_num(mean_melt_bulk_comb[sample]) - np.nan_to_num(std_melt_bulk_comb[sample]), 
                     np.nan_to_num(mean_melt_bulk_comb[sample]) + np.nan_to_num(std_melt_bulk_comb[sample]), 
                     color = colours[sample], alpha = 0.2)
    ax = plt.gca()
    ### DISPLAY HORIZONTAL THRESHOLD USED TO CALCULATE T_m
    midpoint = (np.max(np.nan_to_num(mean_melt_bulk_comb[sample])) + np.min(np.nan_to_num(mean_melt_bulk_comb[sample])))/2
    plt.axhline(midpoint, color = 'black', linestyle = 'dotted')
    # DISPLAY VERTICAL LINE AT FOUND T_m VALUE
    plt.axvline(temp[get_step(sigmoidify(np.nan_to_num(mean_melt_bulk_comb[sample])))], 
                lw = 2.0, linestyle = 'dotted', color = 'black')
    plt.text(midpoint, 
             temp[get_step(sigmoidify(np.nan_to_num(mean_melt_bulk_comb[sample])))], 
             str(temp[get_step(sigmoidify(np.nan_to_num(mean_melt_bulk_comb[sample])))]) + r' °C', 
             fontsize = 20, color = colours[sample], 
             path_effects=[pe.withStroke(linewidth=0.8, foreground="black")])
    #print(r'$T_{m} = $', temp[get_step(sigmoidify(np.nan_to_num(mean_melt_bulk_comb[sample])))], '$^o$C')
    ax.tick_params(direction = 'in', length = 6)
    plt.ylabel(r'$\rm\mu_{CLD}$ [$\rm\mu$m]', fontsize = 20)
    plt.yticks([0, 25, 50, 75, 100, 125, 150], [0, '', 50, '', 100, '', 150], fontsize = 20)
    plt.ylim([-10, 160])
    if ind == 2: 
        plt.xlabel('Temperature [°C]', fontsize = 20)
        plt.xticks([25, 50, 75], [25, 50, 75], fontsize = 20)
    else: 
        plt.xticks([25, 50, 75], [])
    plt.legend(frameon = False, fontsize = 20, loc = 'upper right');
plt.show()

In [ ]:
# Figure 1c - scatter plot of T_Ms (melting temperatures)
# Source: https://matplotlib.org/stable/gallery/subplots_axes_and_figures/broken_axis.html

samples = ['A', 'B', 'C']
mapping = {'A' : 'NS_A', 'B' : 'NS_B', 'C' : 'NS_C'}
melting_temps = [
    temp[get_step(sigmoidify(np.nan_to_num(mean_melt_bulk_comb[mapping[sample]])))] 
    for sample in samples
]
colors = ['orangered', 'cyan', 'silver']

fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True, figsize = (4, 4), gridspec_kw={'height_ratios': [4, 1]})
fig.subplots_adjust(hspace=0.05)  # adjust space between Axes

# plot the same data on both axes
for (sample, melting_temp), color in zip(zip(samples, melting_temps), colors): 
    ax1.plot(sample, melting_temp, color=color, marker='o', ms = 12, mec='black')
    ax2.plot(sample, melting_temp, color=color, marker='o', ms = 12, mec='black')

# limit the view to different portions of the data
ax1.set_ylim(35, 55)  # outliers only
ax2.set_ylim(0, 5)  # most of the data

# hide the spines between ax and ax2
ax1.spines.bottom.set_visible(False)
ax2.spines.top.set_visible(False)
ax1.xaxis.tick_top()
ax1.tick_params(labeltop=False)  # don't put tick labels at the top
ax2.xaxis.tick_bottom()

# Make slanted lines to indicate discontinuous y-axis
d = .5  # proportion of vertical to horizontal extent of the slanted line
kwargs = dict(marker=[(-1, -d), (1, d)], markersize=12,
              linestyle="none", color='k', mec='k', mew=1, clip_on=False)
ax1.plot([0, 1], [0, 0], transform=ax1.transAxes, **kwargs)
ax2.plot([0, 1], [1, 1], transform=ax2.transAxes, **kwargs)

ax1.tick_params(direction = 'in', length = 6)
ax2.tick_params(direction = 'in', length = 6)
ax2.set_xticklabels(['A', 'B', 'C'], fontsize = 25); 
ax1.set_yticks([40, 45, 50])
ax1.set_yticklabels([40, 45, 50], fontsize = 25); 
ax2.set_yticks([0])
ax2.set_yticklabels([0], fontsize = 25); 
ax1.set_xlim([-0.25, 2.25])
ax2.set_xlabel('RNA Construct', fontsize = 25)
ax1.set_ylabel(r'T$\rm_M$ [°C]', fontsize = 25);
plt.show()